# Análisis de Redes Complejas para la Identificación de Patrones de Comportamiento en las Ventas

In [1]:
import pandas as pd
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from networkx.algorithms import community
import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
import cudf

/home/insightlab/anaconda3/envs/similarity/lib/python3.10/site-packages/cupy/_environment.py:540: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [3]:
# Cargar el dataframe de ofertas relámpago
df_ofertas_relampago = pd.read_csv('/home/insightlab/Documents/MercadoLibre/arquivos/ofertas_relampago.csv', encoding='latin1')
df_ofertas_relampago

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS
3,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-23 01:36:12+00:00,lightning_deal,15,13,7.03,2.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES
4,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 15:48:12+00:00,lightning_deal,15,0,39.65,15.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48741,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:01+00:00,lightning_deal,15,9,16.28,6.0,NaN,none,HOME&DECOR,HOME & INDUSTRY,MLM-CHRISTMAS_LIGHTS
48742,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:01+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-DECORATIVE_PAINTINGS
48743,2021-06-19,2021-06-19 07:00:00+00:00,2021-06-19 13:00:03+00:00,lightning_deal,5,3,16.62,2.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-INDOOR_CURTAINS_AND_BLINDS
48744,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:00+00:00,lightning_deal,5,1,38.79,4.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-INDOOR_CURTAINS_AND_BLINDS


In [4]:
len(df_ofertas_relampago['DOMAIN_ID'].unique())

1266

In [5]:
len(df_ofertas_relampago['OFFER_START_DATE'].unique())

61

## Identificación de Patrones en las Redes de Productos

EN esta sección vamos a identificar estos grupos de productos que tienden a venderse juntos en las ofertas relámpago, la cual permitirá optimizar las estrategias de precio y maximizar márgenes.

In [6]:
# Crear un dataframe con las combinaciones de productos que se venden juntos en la misma oferta
product_pairs = []

for _, group in df_ofertas_relampago.groupby('OFFER_START_DATE'):
    products = group['DOMAIN_ID'].unique()
    if len(products) > 1:
        pairs = list(itertools.combinations(products, 2))
        product_pairs.extend(pairs)

# Convertir a DataFrame
df_pairs = pd.DataFrame(product_pairs, columns=['Product_A', 'Product_B'])

# Contar las veces que cada par de productos aparece juntos
df_pairs_count = df_pairs.groupby(['Product_A', 'Product_B']).size().reset_index(name='wdegree')

# Construcción de la red
G = nx.Graph()

# Añadir los nodos y aristas (enlaces)
for _, row in df_pairs_count.iterrows():
    G.add_edge(row['Product_A'], row['Product_B'], weight=row['wdegree'])

In [7]:
df_pairs_count

,Product_A,Product_B,wdegree
0,MLM-3D_PENS,MLM-ACTION_FIGURES,1
1,MLM-3D_PENS,MLM-ACUPUNCTURE_NEEDLES,1
2,MLM-3D_PENS,MLM-ADHESIVE_TAPES,1
3,MLM-3D_PENS,MLM-AEROBICS_AND_FITNESS_EQUIPMENT,1
4,MLM-3D_PENS,MLM-AIRGUN_PELLETS,1
...,...,...,...
363458,MLM-YOGA_MATS,MLM-WORKBENCHES,1
363459,MLM-YOGA_MATS,MLM-WORKOUT_BENCHES,6
363460,MLM-YOGA_MATS,MLM-WORK_COVERALLS_AND_OVERALLS,1
363461,MLM-YOGA_MATS,MLM-WORK_SCRUBS,9


Los algoritmos como Louvain o Girvan-Newman nos permite detectar comunidades dentro de la red. Estas comunidades pueden representar grupos de productos que se deben considerar para promociones conjuntas.

In [8]:
# Identificación de comunidades
communities = community.louvain_communities(G, weight='weight')
print("Comunidades identificadas:")
for i, community in enumerate(communities):
    print(f"Comunidad {i+1}: {community}")

Comunidades identificadas:
Comunidad 1: {'MLM-TACTICAL_BELTS', 'MLM-BACK_GLASS_WIPER_ARMS', 'MLM-HAND_TOWELS', 'MLM-RECIPROCATING_SAWS', 'MLM-EXTERNAL_LAPTOP_COOLERS', 'MLM-MEDICAL_TOURNIQUETS', 'MLM-MOTORCYCLE_GOGGLES', 'MLM-WIRE_STRIPPERS', 'MLM-NAIL_ART_BRUSHES', 'MLM-HABERDASHERY_BEADS', 'MLM-MANUAL_HOISTS', 'MLM-QUILTS_AND_COVERLETS', 'MLM-FOOD_SLICERS', 'MLM-PET_BRUSHES', 'MLM-TEA_INFUSERS', 'MLM-GARDEN_CHAIRS', 'MLM-HAND_AND_FOOT_MASKS', 'MLM-BABY_CARRIERS', 'MLM-SMARTWATCH_CHARGERS', 'MLM-CRIMPING_PLIERS', 'MLM-GARDENING_AND_AGRICULTURE_SUBSTRATES', 'MLM-NETWORK_PLUG_COVERS', 'MLM-TRAMPOLINES', 'MLM-BIRD_FOODS', 'MLM-KITCHEN_TONGS', 'MLM-SCHOOL_KITS', 'MLM-DESKTOP_COMPUTER_CASES', 'MLM-ELECTRIC_WAFFLE_MAKERS', 'MLM-UNDERWEAR_ORGANIZERS', 'MLM-SKIRTS', 'MLM-HAZELNUT_CREAMS', 'MLM-MEMORY_CARD_READERS', 'MLM-RECTIFIER_DIODES', 'MLM-TEQUILAS', 'MLM-DOG_LEASHES', 'MLM-SLATWALL_PANEL_SUPPORTS', 'MLM-DIVING_MASKS', 'MLM-SPORT_AND_MEDICAL_BANDAGES', 'MLM-3D_PRINTERS', 'MLM-RESPIRATORS'

Por ejemplo, productos relacionados con utensilios de cocina, ropa deportiva, y accesorios de dispositivos electrónicos suelen agruparse en comunidades específicas. Esto sugiere que estos productos pueden tener una demanda complementaria y, por lo tanto, podrían beneficiarse de estrategias de bundling (agrupación en paquetes de ofertas) o promociones conjuntas.

### Evaluación de la Importancia Financiera

Los productos que tienen alta centralidad en la red pueden ser fundamentales para la estrategia de precios.

In [9]:
# Calcular la centralidad de intermediación
betweenness = nx.betweenness_centrality(G, weight='weight')
sorted_by_betweenness = sorted(betweenness.items(), key=lambda item: item[1], reverse=True)
top_10_critical_nodes = sorted_by_betweenness[:10]
print("Top 10 nodos más críticos según centralidad de intermediación:")
top_10_critical_nodes

Top 10 nodos más críticos según centralidad de intermediación:


[('MLM-BATH_TOWELS', 0.004205548394982671),
 ('MLM-FITNESS_TRAMPOLINES', 0.0029132113769020367),
 ('MLM-BEAUTY_AND_PERSONAL_CARE_SUPPLIES', 0.0025918163435676733),
 ('MLM-VEHICLE_ACCESSORIES', 0.0025167352990808703),
 ('MLM-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS', 0.0024168814301443685),
 ('MLM-DOG_CARRIERS_AND_CARRYING_BAGS', 0.0023807649617991304),
 ('MLM-SPORT_BRAS', 0.00232335190804214),
 ('MLM-SMARTWATCHES', 0.0022779042409188173),
 ('MLM-BABY_SUPPLIES', 0.002276953037231563),
 ('MLM-FLATWARE_SETS', 0.0022667974513376284)]

Los productos como MLM-BATH_TOWELS y MLM-FITNESS_TRAMPOLINES tienen alta centralidad de intermediación, lo que significa que son productos clave en la red de ventas. Estos productos actúan como puentes en la red, conectando diferentes grupos de productos. Un cambio en el precio o promoción de estos productos podría tener un impacto significativo en las ventas de otros productos conectados. Sería interesante crear escenarios de simulación de cambio de precios para poder observar el comportamient de la red.

In [10]:
# Calcular la centralidad de grado
degree_centrality = nx.degree_centrality(G)
sorted_by_degree = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
top_10_degree_nodes = sorted_by_degree[:10]
print("Top 10 nodos más críticos según centralidad de grado:")
top_10_degree_nodes

Top 10 nodos más críticos según centralidad de grado:


[('MLM-PANTS', 1.0),
 ('MLM-SHORTS', 1.0),
 ('MLM-SURGICAL_AND_INDUSTRIAL_MASKS', 1.0),
 ('MLM-WALLETS', 1.0),
 ('MLM-HEADPHONES', 1.0),
 ('MLM-SANDALS_AND_FLIP_FLOPS', 1.0),
 ('MLM-WRISTWATCHES', 1.0),
 ('MLM-SNEAKERS', 0.9984189723320158),
 ('MLM-AEROBICS_AND_FITNESS_EQUIPMENT', 0.9976284584980237),
 ('MLM-ACTION_FIGURES', 0.9960474308300395)]

In [11]:
# Calcular la centralidad de cercanía
closeness_centrality = nx.closeness_centrality(G)
sorted_by_closeness = sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)
top_10_closeness_nodes = sorted_by_closeness[:10]
print("Top 10 nodos más críticos según centralidad de cercanía:")
top_10_closeness_nodes

Top 10 nodos más críticos según centralidad de cercanía:


[('MLM-PANTS', 1.0),
 ('MLM-SHORTS', 1.0),
 ('MLM-SURGICAL_AND_INDUSTRIAL_MASKS', 1.0),
 ('MLM-WALLETS', 1.0),
 ('MLM-HEADPHONES', 1.0),
 ('MLM-SANDALS_AND_FLIP_FLOPS', 1.0),
 ('MLM-WRISTWATCHES', 1.0),
 ('MLM-SNEAKERS', 0.9984214680347278),
 ('MLM-AEROBICS_AND_FITNESS_EQUIPMENT', 0.9976340694006309),
 ('MLM-ACTION_FIGURES', 0.9960629921259843)]

Los roductos como MLM-PANTS, MLM-SHORTS, y MLM-SURGICAL_AND_INDUSTRIAL_MASKS tienen alta centralidad de grado y cercanía, lo que indica que están altamente conectados con muchos otros productos. Estos productos podrían ser cruciales para campañas de marketing debido a su amplia conectividad en la red, osea productos comúnmente vendido junto con otros.

### Simulación de Escenarios para la Optimización de Márgenes de Ganancia

Vamos aplicar un modelo predictivo para predecir las ventas con y sin descuentos, y considerar la venta cruzada y bundling para maximizar las ganancias

In [12]:
# Simulación de ajustes de precio
def simulate_price_impact(node, price_change, G, df):
    # Ajustar el precio de un producto y evaluar el impacto en la red
    affected_products = [edge[1] for edge in G.edges(node)]
    impact = 0
    for product in affected_products:
        # Supongamos que la elasticidad precio-demanda es inversa
        impact += price_change * (1 / (1 + G[node][product]['weight']))
    
    # Simular el impacto en las ventas globales
    total_sales_impact = impact * len(affected_products)
    print(f"Impacto estimado en ventas por cambiar el precio de {node}: {total_sales_impact}")
    return total_sales_impact

# Cambiamos el precio de los productos más críticos y evaluar el impacto
for node, _ in top_10_critical_nodes:
    simulate_price_impact(node, 0.1, G, df_ofertas_relampago)  # Aumentar el precio en un 10%

Impacto estimado en ventas por cambiar el precio de MLM-BATH_TOWELS: 43981.51114330088
Impacto estimado en ventas por cambiar el precio de MLM-FITNESS_TRAMPOLINES: 41604.63730519478
Impacto estimado en ventas por cambiar el precio de MLM-BEAUTY_AND_PERSONAL_CARE_SUPPLIES: 48010.570646464716
Impacto estimado en ventas por cambiar el precio de MLM-VEHICLE_ACCESSORIES: 46975.74474738165
Impacto estimado en ventas por cambiar el precio de MLM-AUDIO_AND_VIDEO_CABLES_AND_ADAPTERS: 39675.15629778872
Impacto estimado en ventas por cambiar el precio de MLM-DOG_CARRIERS_AND_CARRYING_BAGS: 35746.897346709215
Impacto estimado en ventas por cambiar el precio de MLM-SPORT_BRAS: 28595.70944444467
Impacto estimado en ventas por cambiar el precio de MLM-SMARTWATCHES: 45252.50327238228
Impacto estimado en ventas por cambiar el precio de MLM-BABY_SUPPLIES: 44228.97088625507
Impacto estimado en ventas por cambiar el precio de MLM-FLATWARE_SETS: 30930.501849458


El impacto estimado en ventas por un cambio del 10% en el precio de los productos más críticos. Por ejemplo, se estima que un cambio en el precio de MLM-BEAUTY_AND_PERSONAL_CARE_SUPPLIES podría tener un impacto en ventas de aproximadamente 48,010 unidades. Este análisis es crucial para entender cómo las decisiones de precios pueden influir en la rentabilidad de toda la red de productos.

# Estrategia para Implementar Modelos Predictivos con Venta Cruzada (cross-selling) y Bundling

 Al ofrecer bundles (creación de paquetes de productos que ofrezcan valor añadido al cliente mientras maximizan la ganancia) o estrategias de venta cruzada basadas en patrones reales de compra, se puede mejorar la experiencia del cliente y aumentar el ticket promedio, optimizando así los márgenes de ganancia.



In [13]:
features = [
    'VERTICAL', 'SHIPPING_PAYMENT_TYPE', 'DOMAIN_ID',
    'INVOLVED_STOCK', 'REMAINING_STOCK_AFTER_END', 'SOLD_AMOUNT', 'OFFER_DURATION_HOURS',
    'OFFER_START_DAY', 'OFFER_START_MONTH', 'OFFER_START_WEEKDAY'
]

In [14]:
# Añadir las centralidades como nuevas columnas en el dataframe original
df_ofertas_relampago['betweenness_centrality'] = df_ofertas_relampago['DOMAIN_ID'].map(betweenness)
df_ofertas_relampago['degree_centrality'] = df_ofertas_relampago['DOMAIN_ID'].map(degree_centrality)
df_ofertas_relampago['closeness_centrality'] = df_ofertas_relampago['DOMAIN_ID'].map(closeness_centrality)


In [15]:
df_ofertas_relampago

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,betweenness_centrality,degree_centrality,closeness_centrality
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS,0.000555,0.471146,0.654085
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS,0.001154,0.783399,0.821962
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS,0.001908,0.977075,0.977589
3,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-23 01:36:12+00:00,lightning_deal,15,13,7.03,2.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES,0.001800,1.000000,1.000000
4,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 15:48:12+00:00,lightning_deal,15,0,39.65,15.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES,0.001800,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48741,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:01+00:00,lightning_deal,15,9,16.28,6.0,NaN,none,HOME&DECOR,HOME & INDUSTRY,MLM-CHRISTMAS_LIGHTS,0.000621,0.743874,0.796098
48742,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:01+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-DECORATIVE_PAINTINGS,0.000890,0.734387,0.790131
48743,2021-06-19,2021-06-19 07:00:00+00:00,2021-06-19 13:00:03+00:00,lightning_deal,5,3,16.62,2.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-INDOOR_CURTAINS_AND_BLINDS,0.001500,0.935178,0.939124
48744,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:00+00:00,lightning_deal,5,1,38.79,4.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-INDOOR_CURTAINS_AND_BLINDS,0.001500,0.935178,0.939124


## Probabilidad de Compra 
La idea detrás de la probabilidad de compra es predecir si un cliente comprará un producto adicional dado que ya ha comprado otro producto.

In [16]:
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score, f1_score

Generar todas las combinaciones posibles de productos:

In [17]:
def create_all_product_pairs(df):
    all_products = df['DOMAIN_ID'].unique()
    product_pairs = list(combinations(all_products, 2))
    return pd.DataFrame(product_pairs, columns=['Product_A', 'Product_B'])

df_pairs = create_all_product_pairs(df_ofertas_relampago)

In [18]:
len(df_pairs)

800745

Añadimos una columna que indique si ambos productos del par fueron comprados juntos (1) o no (0).

In [ ]:
def check_purchased_together(row, df):
    purchases_A = df[df['DOMAIN_ID'] == row['Product_A']]['OFFER_START_DATE']
    purchases_B = df[df['DOMAIN_ID'] == row['Product_B']]['OFFER_START_DATE']
    # Check if there is any overlap in the offer dates
    purchased_together = purchases_A.isin(purchases_B).any()
    return 1 if purchased_together else 0

df_pairs['Purchased_Together'] = df_pairs.apply(check_purchased_together, df=df_ofertas_relampago, axis=1)

In [ ]:
df_pairs

,Product_A,Product_B,Purchased_Together
0,MLM-BIRD_FOODS,MLM-ANIMAL_AND_PET_PRODUCTS,1
1,MLM-BIRD_FOODS,MLM-SPEAKERS,1
2,MLM-BIRD_FOODS,MLM-HEADPHONES,1
3,MLM-BIRD_FOODS,MLM-SMART_SPEAKERS,1
4,MLM-BIRD_FOODS,MLM-ELECTRONIC_PRODUCTS,1
...,...,...,...
800740,MLM-THERMAL_REFRIGERATORS_AND_BAGS,MLM-PHOTOGRAPHY_ACCESSORIES_AND_SPARE_PARTS,1
800741,MLM-THERMAL_REFRIGERATORS_AND_BAGS,MLM-ELECTRIC_SANDWICH_MAKERS,1
800742,MLM-AEROBIC_CRUNCH_MACHINES,MLM-PHOTOGRAPHY_ACCESSORIES_AND_SPARE_PARTS,1
800743,MLM-AEROBIC_CRUNCH_MACHINES,MLM-ELECTRIC_SANDWICH_MAKERS,1


Definimos nuestros features con aquellas variables que puedan influir en la probabilidad de que dos productos sean comprados juntos:

*   **Cantidad de veces que cada producto fue vendido:**
    *   SOLD_QUANTITY_A
    *   SOLD_QUANTITY_B
*   **Monto total vendido para cada producto:**
    *   SOLD_AMOUNT_A
    *   SOLD_AMOUNT_B
*   **Categoría del producto (VERTICAL):**
    *   VERTICAL_A
    *   VERTICAL_B
*   **Centralidad en la red de productos:**
    *   betweenness_centrality_A
    *   betweenness_centrality_B
    *   degree_centrality_A
    *   degree_centrality_B
    *   closeness_centrality_A
    *   closeness_centrality_B

In [ ]:
# Extraer las características individuales de los productos
df_product_features = df_ofertas_relampago[['DOMAIN_ID', 'VERTICAL', 'SOLD_QUANTITY', 'SOLD_AMOUNT', 
                                            'betweenness_centrality', 'degree_centrality', 'closeness_centrality']]

# Renombrar las columnas para Producto A
df_product_features_A = df_product_features.rename(columns={
    'DOMAIN_ID': 'Product_A',
    'VERTICAL': 'VERTICAL_A',
    'SOLD_QUANTITY': 'SOLD_QUANTITY_A',
    'SOLD_AMOUNT': 'SOLD_AMOUNT_A',
    'betweenness_centrality': 'betweenness_centrality_A',
    'degree_centrality': 'degree_centrality_A',
    'closeness_centrality': 'closeness_centrality_A'
})

# Renombrar las columnas para Producto B
df_product_features_B = df_product_features.rename(columns={
    'DOMAIN_ID': 'Product_B',
    'VERTICAL': 'VERTICAL_B',
    'SOLD_QUANTITY': 'SOLD_QUANTITY_B',
    'SOLD_AMOUNT': 'SOLD_AMOUNT_B',
    'betweenness_centrality': 'betweenness_centrality_B',
    'degree_centrality': 'degree_centrality_B',
    'closeness_centrality': 'closeness_centrality_B'
})

: 

In [ ]:
# Unir la información de Producto A y Producto B con los pares generados
df_training = df_pairs.merge(df_product_features_A, on='Product_A')
df_training = df_training.merge(df_product_features_B, on='Product_B')

In [ ]:
# Crear columnas adicionales para características combinadas
df_training['Same_Vertical'] = (df_training['VERTICAL_A'] == df_training['VERTICAL_B']).astype(int)

# Por ejemplo, similitud entre centralidades
df_training['Centrality_Difference'] = abs(df_training['degree_centrality_A'] - df_training['degree_centrality_B'])

In [ ]:
df_training

In [ ]:
# Definir las columnas categóricas que queremos codificar
categorical_features = ['VERTICAL_A', 'VERTICAL_B', 'DOMAIN_ID_A', 'DOMAIN_ID_B']

# Crear el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], remainder='passthrough')

In [ ]:
# Definir la variable objetivo
target = 'Purchased_Together'

In [ ]:
df_training

In [ ]:
X = df_training[features]
y = df_training[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Modelos aplicados a la predicción de compras Bundling

Aplicar Random Forest

In [ ]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

rf_param_grid = {
    'model__n_estimators': [500, 1000, 1500],
    'model__max_depth': [4, 6, 8],
    'model__min_samples_split': [2, 5, 10]
}
# Configurar GridSearchCV para Random Forest
rf_grid = GridSearchCV(estimator=rf_pipeline, param_grid=rf_param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
rf_grid.fit(X_train, y_train)

In [ ]:
# Obtener los mejores parámetros
best_params_rf = rf_grid.best_params_
print("Mejores parámetros encontrados para Random Forest Regressor:", best_params_rf)

Aplicar LogisticRegression

In [ ]:
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(random_state=42))
])

lr_param_grid = {
    'model__C': [0.1, 1, 10],
    'model__penalty': ['l2'],
    'model__solver': ['lbfgs']
}
lr_grid = GridSearchCV(estimator=lr_pipeline, param_grid=lr_param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
lr_grid.fit(X_train, y_train)

In [ ]:
# Obtener los mejores parámetros
best_params_lr = lr_grid.best_params_
print("Mejores parámetros encontrados para LogisticRegression:", best_params_lr)

Aplicar XGBoost

In [ ]:
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier(random_state=42, use_label_encoder=False))
])

xgb_param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [4, 6, 8],
    'model__learning_rate': [0.01, 0.1, 0.2]
}
xgb_grid = GridSearchCV(estimator=xgb_pipeline, param_grid=xgb_param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
xgb_grid.fit(X_train, y_train)

In [ ]:
# Obtener los mejores parámetros
best_params_xgb = xgb_grid.best_params_
print("Mejores parámetros encontrados para XGBoost:", best_params_xgb)

Aplicar Support Vector Machine (SVM)

In [ ]:
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', SVC(random_state=42, probability=True))
])

svm_param_grid = {
    'model__C': [0.1, 1, 10],
    'model__kernel': ['linear', 'rbf'],
    'model__gamma': ['scale', 'auto']
}
svm_grid = GridSearchCV(estimator=svm_pipeline, param_grid=svm_param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
svm_grid.fit(X_train, y_train)

In [ ]:
# Obtener los mejores parámetros
best_params_svm = svm_grid.best_params_
print("Mejores parámetros encontrados para Support Vector Machine (SVM):", best_params_svm)

In [ ]:
# Comparación de los mejores resultados
print("Best RF AUC: ", rf_grid.best_score_)
print("Best LR AUC: ", lr_grid.best_score_)
print("Best XGB AUC: ", xgb_grid.best_score_)
print("Best SVM AUC: ", svm_grid.best_score_)

In [ ]:
# Predecir en el conjunto de prueba
y_test_pred_rf = rf_grid.predict(X_test)
y_test_pred_lr = lr_grid.predict(X_test)
y_test_pred_xgb = xgb_grid.predict(X_test)
y_test_pred_svm = svm_grid.predict(X_test)

In [ ]:
# Función para calcular métricas
def calculate_metrics(y_true, y_pred):
    y_pred_rounded = np.round(y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    accuracy = accuracy_score(y_true, y_pred_rounded)
    recall = recall_score(y_true, y_pred_rounded, average='weighted')
    precision = precision_score(y_true, y_pred_rounded, average='weighted')
    f1 = f1_score(y_true, y_pred_rounded, average='weighted')
    return rmse, accuracy, recall, precision, f1

# Calcular métricas para cada modelo
metrics_rf = calculate_metrics(y_test, y_test_pred_rf)
metrics_lr = calculate_metrics(y_test, y_test_pred_lr)
metrics_xgb = calculate_metrics(y_test, y_test_pred_xgb)
metrics_svm = calculate_metrics(y_test, y_test_pred_svm)

# Crear un DataFrame con los resultados de las métricas
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 'XGBoost', 'SVM'],
    'RMSE': [metrics_rf[0], metrics_lr[0], metrics_xgb[0], metrics_svm[0]],
    'Accuracy': [metrics_rf[1], metrics_lr[1], metrics_xgb[1], metrics_svm[1]],
    'Recall': [metrics_rf[2], metrics_lr[2], metrics_xgb[2], metrics_svm[2]],
    'Precision': [metrics_rf[3], metrics_lr[3], metrics_xgb[3], metrics_svm[3]],
    'F1_Score': [metrics_rf[4], metrics_lr[4], metrics_xgb[4], metrics_svm[4]]
})

# Mostrar el DataFrame con la comparación
comparison_df

In [ ]:
# Crear un DataFrame para comparar los valores reales con las predicciones de cada modelo
comparison_df = pd.DataFrame({
    'Real': y_test,
    'Random Forest': y_test_pred_rf,
    'Logistic Regression': y_test_pred_lr,
    'XGBoost': y_test_pred_xgb,
    'SVM': y_test_pred_svm
})

# Mostrar las primeras 20 filas para comparación
comparison_df.head(20)

In [ ]:
# Gráfico de dispersión para ver las predicciones vs valores reales
plt.figure(figsize=(20, 8))

# Random Forest
plt.subplot(2, 2, 1)
sns.scatterplot(x=comparison_df['Real'], y=comparison_df['Random Forest Predicción'])
plt.title('Random Forest: Real vs Predicción')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')

# Logistic Regression
plt.subplot(2, 2, 2)
sns.scatterplot(x=comparison_df['Real'], y=comparison_df['Logistic Regression Predicción'])
plt.title('Logistic Regression: Real vs Predicción')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')

# XGBoost
plt.subplot(2, 2, 3)
sns.scatterplot(x=comparison_df['Real'], y=comparison_df['XGBoost Predicción'])
plt.title('XGBoost: Real vs Predicción')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')

# SVM
plt.subplot(2, 2, 4)
sns.scatterplot(x=comparison_df['Real'], y=comparison_df['SVM Predicción'])
plt.title('SVM: Real vs Predicción')
plt.xlabel('Valor Real')
plt.ylabel('Predicción')

plt.tight_layout()
plt.show()